In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import the python file
import importlib, banyan_sigma
importlib.reload(banyan_sigma)

#import the main function from the file
from banyan_sigma import banyan_sigma

I'm using Gaia data that Beck pulled for the distance re-estimates. The file happened to have everything I needed—ra, dec, proper motion in ra and dec, errors in proper motion, parallax—so I'm directly using that.

It resolved ~360 of the stars in the database.

In [6]:
# read in csv which i originally made in a separate folder then moved here
stars = pd.read_csv('follette_banyansigma_input.csv')
# rename the columns so they're how banyan sigma likes it
stars.columns = ['','RA','DEC','PMRA','PMDEC','EPMRA','EPMDEC','RV','ERV','PLX','EPLX']

#remove rows with negative parallax values, which are clearly unphysical
#(removes 6 negative plx values and 28 with nan values)
stars = stars[stars['PLX']>0]

# remove rv (only 13 not nan) and drop remaining nans (actually, none left)
stars_data = stars[['RA','DEC','PMRA','PMDEC','EPMRA','EPMDEC','PLX','EPLX']].dropna()

In [7]:
#take a look at the data:
#we'll recover 327 stars
stars_data

,RA,DEC,PMRA,PMDEC,EPMRA,EPMDEC,PLX,EPLX
0,242.248026,-38.941100,-9.872,-23.526,0.465,0.300,6.6565,0.2796
1,235.419938,-33.755330,-19.095,-21.107,0.575,0.405,6.6964,0.3486
2,242.155490,-39.386472,-12.460,-24.016,1.798,0.873,5.3478,0.8740
3,246.589703,-24.433720,-9.050,-25.287,0.403,0.250,7.3210,0.1942
4,246.777442,-24.697020,-5.694,-25.030,0.291,0.197,7.0362,0.1327
...,...,...,...,...,...,...,...,...
356,166.745484,-77.314828,-23.319,1.714,0.097,0.097,5.2538,0.0576
357,163.998701,-77.411146,-23.813,2.193,0.059,0.053,5.4030,0.0312
358,165.465914,-34.704791,-68.389,-14.016,0.054,0.059,16.6428,0.0416
359,240.111213,-20.942191,-10.546,-17.225,0.481,0.323,6.1617,0.2008


We get an error here: KeyError with an astropy.tables object. VCA not found. Looking into it but haven't figured it out.

In [8]:
membership_probs = banyan_sigma(stars_data=stars_data,use_plx=True)

KeyError: 'No matches found for key VCA'